In [ ]:
class ElevatorStateMachine:
    def __init__(self, elevator):
        self.elevator = elevator
        self.states = {floor: FloorState(floor, self) for floor in range(1, elevator.floors + 1)}
        self.current_state = self.states[elevator.current_floor]

    def set_state(self, floor):
        self.current_state = self.states[floor]
        self.elevator.current_floor = floor
        self.elevator.commands.append(f"Лифт находится на {floor} этаже")


class FloorState:
    def __init__(self, floor, machine):
        self.floor = floor
        self.machine = machine
        self.setup_transitions()

    def setup_transitions(self):
        if self.floor < self.machine.elevator.floors:
            self.move_up = lambda: self.machine.set_state(self.floor + 1)
        if self.floor > 1:
            self.move_down = lambda: self.machine.set_state(self.floor - 1)

    def open_doors(self):
        self.machine.elevator.commands.append(f"Открыть двери на этаже {self.floor}")

    def close_doors(self):
        self.machine.elevator.commands.append(f"Закрыть двери на этаже {self.floor}")


class Elevator:
    def __init__(self, elevator_id, current_floor, floors):
        self.elevator_id = elevator_id
        self.current_floor = current_floor
        self.floors = floors
        self.commands = []
        self.state_machine = ElevatorStateMachine(self)


class Building:
    def __init__(self, floors, elevators):
        self.floors = floors
        self.elevators = elevators

    def call_elevator(self, call_floor, destination_floor):
        distances = [abs(elevator.current_floor - call_floor) for elevator in self.elevators]
        selected_elevator = self.elevators[distances.index(min(distances))]

        while selected_elevator.current_floor != call_floor:
            move_method = {
                True: "move_up",
                False: "move_down"
            }[selected_elevator.current_floor < call_floor]
            getattr(selected_elevator.state_machine.current_state, move_method)()

        selected_elevator.state_machine.current_state.open_doors()
        selected_elevator.state_machine.current_state.close_doors()

        while selected_elevator.current_floor != destination_floor:
            move_method = {
                True: "move_up",
                False: "move_down"
            }[selected_elevator.current_floor < destination_floor]
            getattr(selected_elevator.state_machine.current_state, move_method)()

        selected_elevator.state_machine.current_state.open_doors()
        selected_elevator.state_machine.current_state.close_doors()


max_floors = 10
elevator1 = Elevator(elevator_id=1, current_floor=1, floors=max_floors)
elevator2 = Elevator(elevator_id=2, current_floor=5, floors=max_floors)

building = Building(floors=max_floors, elevators=[elevator1, elevator2])

building.call_elevator(call_floor=5, destination_floor=7)
building.call_elevator(call_floor=3, destination_floor=9)

for elevator in building.elevators:
    print(f"Команды лифта {elevator.elevator_id}:")
    for command in elevator.commands:
        print(command)


Команды лифта 1:
Лифт находится на 2 этаже
Лифт находится на 3 этаже
Открыть двери на этаже 3
Закрыть двери на этаже 3
Лифт находится на 4 этаже
Лифт находится на 5 этаже
Лифт находится на 6 этаже
Лифт находится на 7 этаже
Лифт находится на 8 этаже
Лифт находится на 9 этаже
Открыть двери на этаже 9
Закрыть двери на этаже 9
Команды лифта 2:
Открыть двери на этаже 5
Закрыть двери на этаже 5
Лифт находится на 6 этаже
Лифт находится на 7 этаже
Открыть двери на этаже 7
Закрыть двери на этаже 7
